In [1]:
import pyspark

In [12]:
from pyspark.sql import SQLContext,SparkSession
from pyspark import SparkContext,SparkFiles
spark = SparkSession.builder.master("local[2]").appName("ML_Program").getOrCreate()
sc = spark.sparkContext

In [13]:
url = "https://raw.githubusercontent.com/guru99-edu/R-Programming/master/adult_data.csv"
sc.addFile(url)

In [14]:
sqlContext = SQLContext(sc)
df = sqlContext.read.csv(SparkFiles.get("adult_data.csv"),inferSchema=True,header=True)

In [15]:
df.show(2)

+---+---+---------+------+---------+---------------+------------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+
|  x|age|workclass|fnlwgt|education|educational-num|    marital-status|       occupation|relationship| race|gender|capital-gain|capital-loss|hours-per-week|native-country|income|
+---+---+---------+------+---------+---------------+------------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+
|  1| 25|  Private|226802|     11th|              7|     Never-married|Machine-op-inspct|   Own-child|Black|  Male|           0|           0|            40| United-States| <=50K|
|  2| 38|  Private| 89814|  HS-grad|              9|Married-civ-spouse|  Farming-fishing|     Husband|White|  Male|           0|           0|            50| United-States| <=50K|
+---+---+---------+------+---------+---------------+------------------+-----------------+------------+---

In [16]:
from pyspark.ml.feature import StringIndexer,OneHotEncoder
si = StringIndexer(inputCol="workclass",outputCol="workclass_encoded")
model = si.fit(df)
df = model.transform(df)
df.printSchema()

root
 |-- x: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- educational-num: integer (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- capital-gain: integer (nullable = true)
 |-- capital-loss: integer (nullable = true)
 |-- hours-per-week: integer (nullable = true)
 |-- native-country: string (nullable = true)
 |-- income: string (nullable = true)
 |-- workclass_encoded: double (nullable = false)



In [18]:
df.show(20)

+---+---+----------------+------+------------+---------------+------------------+-----------------+-------------+------------------+------+------------+------------+--------------+--------------+------+-----------------+
|  x|age|       workclass|fnlwgt|   education|educational-num|    marital-status|       occupation| relationship|              race|gender|capital-gain|capital-loss|hours-per-week|native-country|income|workclass_encoded|
+---+---+----------------+------+------------+---------------+------------------+-----------------+-------------+------------------+------+------------+------------+--------------+--------------+------+-----------------+
|  1| 25|         Private|226802|        11th|              7|     Never-married|Machine-op-inspct|    Own-child|             Black|  Male|           0|           0|            40| United-States| <=50K|              0.0|
|  2| 38|         Private| 89814|     HS-grad|              9|Married-civ-spouse|  Farming-fishing|      Husband|   

In [25]:
df.groupby("native-country").count().sort("count",ascending=True).show(2)

+------------------+-----+
|    native-country|count|
+------------------+-----+
|Holand-Netherlands|    1|
|           Hungary|   19|
+------------------+-----+
only showing top 2 rows



In [35]:
df.filter(df["native-country"] == 'Holand-Netherlands').count()

0

In [33]:
df = df.filter(df["native-country"] != "Holand-Netherlands")

In [46]:
df.count()

48841

In [47]:
df.show(2)

+---+---+---------+------+---------+---------------+------------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+-----------------+
|  x|age|workclass|fnlwgt|education|educational-num|    marital-status|       occupation|relationship| race|gender|capital-gain|capital-loss|hours-per-week|native-country|income|workclass_encoded|
+---+---+---------+------+---------+---------------+------------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+-----------------+
|  1| 25|  Private|226802|     11th|              7|     Never-married|Machine-op-inspct|   Own-child|Black|  Male|           0|           0|            40| United-States| <=50K|              0.0|
|  2| 38|  Private| 89814|  HS-grad|              9|Married-civ-spouse|  Farming-fishing|     Husband|White|  Male|           0|           0|            50| United-States| <=50K|              0.0|
+---+---+------

In [45]:
stringIndexer = StringIndexer(inputCol="workclass", outputCol="workclass_encoded1")
model = stringIndexer.fit(df)
indexed = model.transform(df)
encoder = OneHotEncoder(dropLast=False, inputCol="workclass_encoded1", outputCol="workclass_vec")
encoded = encoder.fit(indexed)
enc = encoded.transform(indexed)
enc.show(2)

+---+---+---------+------+---------+---------------+------------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+-----------------+------------------+-------------+
|  x|age|workclass|fnlwgt|education|educational-num|    marital-status|       occupation|relationship| race|gender|capital-gain|capital-loss|hours-per-week|native-country|income|workclass_encoded|workclass_encoded1|workclass_vec|
+---+---+---------+------+---------+---------------+------------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+-----------------+------------------+-------------+
|  1| 25|  Private|226802|     11th|              7|     Never-married|Machine-op-inspct|   Own-child|Black|  Male|           0|           0|            40| United-States| <=50K|              0.0|               0.0|(9,[0],[1.0])|
|  2| 38|  Private| 89814|  HS-grad|              9|Married-civ-spouse|  Farming

In [49]:
df.show(2)

+---+---+---------+------+---------+---------------+------------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+-----------------+
|  x|age|workclass|fnlwgt|education|educational-num|    marital-status|       occupation|relationship| race|gender|capital-gain|capital-loss|hours-per-week|native-country|income|workclass_encoded|
+---+---+---------+------+---------+---------------+------------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+-----------------+
|  1| 25|  Private|226802|     11th|              7|     Never-married|Machine-op-inspct|   Own-child|Black|  Male|           0|           0|            40| United-States| <=50K|              0.0|
|  2| 38|  Private| 89814|  HS-grad|              9|Married-civ-spouse|  Farming-fishing|     Husband|White|  Male|           0|           0|            50| United-States| <=50K|              0.0|
+---+---+------

In [53]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder
CATE_FEATURES = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country']
stages = [] # stages in our Pipeline
for categoricalCol in CATE_FEATURES:
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    print(stringIndexer.getOutputCol(),categoricalCol)
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()],
                                     outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]

workclassIndex workclass
educationIndex education
marital-statusIndex marital-status
occupationIndex occupation
relationshipIndex relationship
raceIndex race
genderIndex gender
native-countryIndex native-country


In [80]:
label_stringIdx =  StringIndexer(inputCol="income", outputCol="new_income")
stages += [label_stringIdx]

In [81]:
CONTI_FEATURES  = ['age', 'fnlwgt','capital-gain', 'educational-num', 'capital-loss', 'hours-per-week']

In [82]:
assemblerInputs = [c + "classVec" for c in CATE_FEATURES] + CONTI_FEATURES

In [83]:
assemblerInputs

['workclassclassVec',
 'educationclassVec',
 'marital-statusclassVec',
 'occupationclassVec',
 'relationshipclassVec',
 'raceclassVec',
 'genderclassVec',
 'native-countryclassVec',
 'age',
 'fnlwgt',
 'capital-gain',
 'educational-num',
 'capital-loss',
 'hours-per-week']

In [78]:
stages.pop(len(stages)-1)

VectorAssembler_4be367a9aec2

In [84]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
print(assembler)
stages += [assembler]

VectorAssembler_b4e26ba3ac1a


In [85]:
stages

[StringIndexer_1e4f2ac13cf6,
 OneHotEncoder_e9c1b1cdcd6e,
 StringIndexer_6db1d649d68c,
 OneHotEncoder_8e5d549da95b,
 StringIndexer_d46a4abc5cb2,
 OneHotEncoder_8c6337367936,
 StringIndexer_4dc612bf009d,
 OneHotEncoder_f8f28f01ec23,
 StringIndexer_2d928ffe73d0,
 OneHotEncoder_c3b5e55929a5,
 StringIndexer_81825a07ddfb,
 OneHotEncoder_12b134972ecc,
 StringIndexer_9507b5ccf022,
 OneHotEncoder_0668462e3782,
 StringIndexer_2a380fd93a3a,
 OneHotEncoder_7fc3268c7135,
 StringIndexer_9a0f69ef58f9,
 VectorAssembler_b4e26ba3ac1a]

In [86]:
pipeline = Pipeline(stages=stages)
pipelinemodel = pipeline.fit(df)
model = pipelinemodel.transform(df)
model.show(1)


+---+---+---------+------+---------+---------------+--------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+-----------------+--------------+-----------------+--------------+-----------------+-------------------+----------------------+---------------+------------------+-----------------+--------------------+---------+-------------+-----------+--------------+-------------------+----------------------+----------+--------------------+
|  x|age|workclass|fnlwgt|education|educational-num|marital-status|       occupation|relationship| race|gender|capital-gain|capital-loss|hours-per-week|native-country|income|workclass_encoded|workclassIndex|workclassclassVec|educationIndex|educationclassVec|marital-statusIndex|marital-statusclassVec|occupationIndex|occupationclassVec|relationshipIndex|relationshipclassVec|raceIndex| raceclassVec|genderIndex|genderclassVec|native-countryIndex|native-countryclassVec|new_income|            featur

In [87]:
model.take(1)

[Row(x=1, age=25, workclass='Private', fnlwgt=226802, education='11th', educational-num=7, marital-status='Never-married', occupation='Machine-op-inspct', relationship='Own-child', race='Black', gender='Male', capital-gain=0, capital-loss=0, hours-per-week=40, native-country='United-States', income='<=50K', workclass_encoded=0.0, workclassIndex=0.0, workclassclassVec=SparseVector(8, {0: 1.0}), educationIndex=5.0, educationclassVec=SparseVector(15, {5: 1.0}), marital-statusIndex=1.0, marital-statusclassVec=SparseVector(6, {1: 1.0}), occupationIndex=6.0, occupationclassVec=SparseVector(14, {6: 1.0}), relationshipIndex=2.0, relationshipclassVec=SparseVector(5, {2: 1.0}), raceIndex=1.0, raceclassVec=SparseVector(4, {1: 1.0}), genderIndex=0.0, genderclassVec=SparseVector(1, {0: 1.0}), native-countryIndex=0.0, native-countryclassVec=SparseVector(40, {0: 1.0}), new_income=0.0, features=SparseVector(99, {0: 1.0, 13: 1.0, 24: 1.0, 35: 1.0, 45: 1.0, 49: 1.0, 52: 1.0, 53: 1.0, 93: 25.0, 94: 22680

In [90]:
from pyspark.ml.linalg import DenseVector
input_data = model.rdd.map(lambda x : (x["new_income"], DenseVector(x["features"])))


In [91]:
feature_df = sqlContext.createDataFrame(input_data,["new_income","features"])

In [93]:
feature_df.show(10)

+----------+--------------------+
|new_income|            features|
+----------+--------------------+
|       0.0|[1.0,0.0,0.0,0.0,...|
|       0.0|[1.0,0.0,0.0,0.0,...|
|       1.0|[0.0,0.0,1.0,0.0,...|
|       1.0|[1.0,0.0,0.0,0.0,...|
|       0.0|[0.0,0.0,0.0,1.0,...|
|       0.0|[1.0,0.0,0.0,0.0,...|
|       0.0|[0.0,0.0,0.0,1.0,...|
|       1.0|[0.0,1.0,0.0,0.0,...|
|       0.0|[1.0,0.0,0.0,0.0,...|
|       0.0|[1.0,0.0,0.0,0.0,...|
+----------+--------------------+
only showing top 10 rows



In [97]:
train,test = feature_df.randomSplit([0.8,0.2],seed=1234)
print("Train data count : {}, Test data count : {}".format(train.count(),test.count()))

Train data count : 39046, Test data count : 9795


In [99]:
train.groupby("new_income").count().sort("new_income",ascending=True).show()

+----------+-----+
|new_income|count|
+----------+-----+
|       0.0|29701|
|       1.0| 9345|
+----------+-----+



In [100]:
test.groupby("new_income").count().sort("new_income",ascending=True).show()

+----------+-----+
|new_income|count|
+----------+-----+
|       0.0| 7453|
|       1.0| 2342|
+----------+-----+



In [107]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol="new_income",featuresCol="features",maxIter=10,regParam=0.3)

In [108]:
model = lr.fit(train)

In [109]:
print("Intercerpt = ",str(model.intercept))

Intercerpt =  -1.934289834031936


In [110]:
predictions = model.transform(test)

In [111]:
predictions.groupby("new_income").count().show()

+----------+-----+
|new_income|count|
+----------+-----+
|       0.0| 7453|
|       1.0| 2342|
+----------+-----+



In [127]:
predictions.printSchema()

root
 |-- new_income: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [114]:
pred_df = predictions.select("new_income","prediction","probability")
pred_df.show(10)

+----------+----------+--------------------+
|new_income|prediction|         probability|
+----------+----------+--------------------+
|       0.0|       0.0|[0.93053780544176...|
|       0.0|       0.0|[0.94603248831750...|
|       0.0|       0.0|[0.81182553768189...|
|       0.0|       0.0|[0.91346022215182...|
|       0.0|       0.0|[0.55398815653954...|
|       0.0|       1.0|[0.28877064190246...|
|       0.0|       1.0|[0.35997365835694...|
|       0.0|       0.0|[0.90778664876464...|
|       0.0|       1.0|[0.44580363687604...|
|       0.0|       1.0|[0.34448851921895...|
+----------+----------+--------------------+
only showing top 10 rows



In [115]:
eval_df = predictions.select("new_income","prediction")
eval_df.filter(eval_df.new_income == eval_df.prediction).count() / eval_df.count()

0.8250127616130679

In [116]:
acc = eval_df.filter(eval_df.new_income == eval_df.prediction).count() / eval_df.count()

In [123]:
print("Accuracy = %.3f%%"%(acc*100))

Accuracy = 82.501%
